In [13]:
import yaml
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt

datapath = "../data/train_half"

files = os.listdir(datapath)

def load_case(i):
    f = h5py.File(f"{datapath}/{files[i]}", "r")
    return f["raw"][:], f["label"][:]

def get_aneurysm_bounds_raw(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = slice(*np.where(x_s)[0][[0, -1]])
    y = slice(*np.where(y_s)[0][[0, -1]])
    z = slice(*np.where(z_s)[0][[0, -1]])

    return x, y, z

def get_aneurysm_center(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = int(np.where(x_s)[0][[0, -1]].mean())
    y = int(np.where(y_s)[0][[0, -1]].mean())
    z = int(np.where(z_s)[0][[0, -1]].mean())

    return x, y, z

def mask_overlay(image, mask, color=(0., 1., 0.), weight=.8):
    """
    Helper function to visualize mask on the top of the aneurysm
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)

    image = np.dstack((image, image, image))
    weighted_sum = image * weight + mask * (1 - weight) # cv2.addWeighted(image, 1 - weight, mask, weight, 0.,  dtype=cv2.CV_32F)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]
    return img

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from ipywidgets import widgets
%matplotlib widget

def mask_overlay(image, mask, color=(0., 1., 0.), weight=.8):
    """
    Helper function to visualize mask on the top of the aneurysm
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = image * weight + mask * (1 - weight) # cv2.addWeighted(image, 1 - weight, mask, weight, 0.,  dtype=cv2.CV_32F)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]
    return img


class ImageSlicer(object):

    def __init__(self, ax, img, mask, start_index):
        self.ax = ax

        # convert to numpy array
        self.image_np = img
        self.mask_np = mask

        # get number of slices
        _, _, self.slices = self.image_np.shape
        self.ind = start_index if start_index is not None else self.slices // 2
        self.show_mask = True

        # plot image with mask overlay
        self.image_plt = self.ax.imshow(self.overlay, vmin=0, vmax=1)
        self._draw()

    @property
    def overlay(self):
        # get image and mask slice
        image = self.image_np[:, :, self.ind]
        image = image / np.max(image)
        image = np.dstack((image, image, image))
        mask = self.mask_np[:, :, self.ind]

        # create masked overlay
        if self.show_mask:
            return mask_overlay(image, mask)
        else: 
            return image

    def onscroll(self, event):

        # get new slice number
        self.ind = event['new']
        self.update()

    def on_show_mask(self, event):

        self.show_mask = event['new']
        with open("file.txt", "a") as f:
            f.write(f"{self.show_mask}\n")
        self.update()

    def update(self):

        # draw overlay
        self.image_plt.set_data(self.overlay)
        self._draw()

    def _draw(self):
        try:
            self.image_plt.axes.figure.canvas.draw(vmin=0, vmax=1)
        except Exception as e:
            with open("file.txt", "a") as f:
                f.write(f"Exception {e}\n")
            pass


def plot3d(img, mask, start_index = None):

    #
    figure, ax = plt.subplots(1, 1)
    tracker = ImageSlicer(ax, img, mask, start_index)

    #
    int_slider = widgets.IntSlider(
        value=tracker.ind,
        min=0,
        max=tracker.slices,
        step=1,
        description='Slice',
        continuous_update=True
    )
    int_slider.observe(tracker.onscroll, 'value')

    show_mask = widgets.Checkbox(
        value=True,
        description='Show Mask',
        disabled=False,
        indent=False
    )
    show_mask.observe(tracker.on_show_mask, 'value')

    return figure, int_slider, show_mask

In [ ]:
from scipy import ndimage as ndi

img, mask = load_case(2)

labels, num_labels = ndi.label(mask)

choosen_coords = []
for i in range(num_labels):
    cur = (labels == i + 1)
    center_coords = get_aneurysm_center(cur)
    choosen_coords.append(center_coords)

idx = np.mgrid[:mask.shape[0], :mask.shape[1], :mask.shape[2]]
ret = np.zeros(mask.shape)
print(idx.shape, ret.shape, mask.shape)
for pic in choosen_coords:

    d = np.sqrt(np.sum((idx - np.array(pic)[:, None, None, None]) ** 2, axis=0))
    d[d == 0] = 1e-10
    t2 = np.clip(1 / d, 0, 1)

    if ret is None:
        ret = t2
    else:
        ret = np.max([ret, t2], axis=0)

ret /= ret.max()

print(choosen_coords)
fig, slider, show_mask = plot3d(ret, ret)

print(ret[:, :, 56].max())

widgets.Box(
    [slider, show_mask]
)



In [4]:
mask_t.shape, img_t.shape

((1, 220, 256, 256), (2, 220, 256, 256))

In [5]:
img_t.max()

1.0

In [6]:
fig, slider, mask_toggle = plot3d(img_t[1], mask_t[0])
slider

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=128, description='Slice', max=256)

In [64]:
pics = [(84, 84, 84), (32, 32, 32)]

idx = np.mgrid[:128, :128, :128]
ret = None
for pic in pics:
    t2 = np.clip(1 / np.sum((idx - np.array(pic)[:, None, None, None]) ** 2, axis=0), 0, 1)

    if ret is None:
        ret = t2
        print(ret.shape)
    else:
        ret = np.max([ret, t2], axis=0)
        print(ret.shape)


# plt.imshow(t2[5])
# np.round(ret[84], 2).max()

# fig, slider, show_mask = plot3d(ret, ret)

# widgets.Box(
#     [slider, show_mask]
# )

(128, 128, 128)
(128, 128, 128)


/home/lm/anaconda3/envs/mlmed/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


1.0

In [13]:
%matplotlib notebook
s = mask.sum(axis=(1, 2))
print(s.max())
np.argmax(s)

plt.imshow(mask[78])

130.0


TypeError: _argmax_dispatcher() missing 1 required positional argument: 'a'